In [1]:
# read in known applicant files, clean and try to dedupe
from collections import Counter
import difflib
from fuzzywuzzy import fuzz
import uuid
import itertools
import pandas as pd
import numpy as np
import string
import funcy
import re
import os


from data_cleaning_functions import correct_mispellings, long_form_date, clean_names, clean_med_school
from dev import (
    SUFFIXES, FEMALE_FIRST_NAMES, NAME_COLS, RAW_ATT_DATA_DIR, ATT_DATA_DIR, PICKLE_DIR, 
        CORRECTIONS_DIR, SUM_STAT_DIR)


%load_ext autoreload
%autoreload 2

%reload_ext autoreload

In [2]:
r1_file = '1964-1973 associates.XLS'
r2_file = 'Associates alpha by institute.XLS'
r3_file = 'Associates data.XLS'
r4_file = 'NIMH Associates Complete.XLS'
r5_file = 'NINDB Associates alpha by year.xls'
filenames = [r1_file, r2_file, r3_file, r4_file, r5_file]

In [3]:
file_df = map(lambda x: pd.read_excel(os.path.join(RAW_ATT_DATA_DIR, x)), filenames)

file_4_columns = [
    'dno', 'source', 'unknown', 'lastname', 'first_middle', 'institute', 'lab_brch', 
    'program', 'supervisor', 'eod_year', 'med_school', 'year_grad', 'intern_hos', 'intern_dte',
       'res_hosp', 'residency', 'res_dtes'
]

file_df[2].rename(columns={'lname':'lastname', 'fname': 'first_middle'}, inplace=True)
file_df[1].rename(columns={'lname':'lastname', 'fname': 'first_middle'}, inplace=True)

file_df[4].columns = file_4_columns
# for each files in the list, add a column to track source
for name, f in zip(filenames, file_df):
    f.loc[:, 'data_source'] = name
concat_df = pd.concat(file_df).reset_index(drop=True)

print sum(map(lambda x: x.shape[0], file_df)) == concat_df.shape[0]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
True


In [4]:
def has_suffix(lst_strs):
    # check for any existing suffix
    existing_suffixes = filter(None, filter(lambda x: x in lst_strs, SUFFIXES))
    # if suffix is I or V, only return suffix if len of lst_string > 2 (otherwise middle name)
    if ('I' in existing_suffixes or 'V' in existing_suffixes) and len(lst_strs) > 2:
        return existing_suffixes
    return funcy.remove(lambda x: x in ['I', 'V', 'DR'], existing_suffixes)

In [5]:
def strip_first_middle(raw_str):
    # seperate first_middle column string (Thomas, Bruce) into THOMAS BRUCE
    # suffix also appears to be in column THOMAS BRUCE JR
    names = filter(None, raw_str.split(' '))
    cleaned_vals = map(clean_names, names)
    suff = has_suffix(cleaned_vals)
    # suffix is actually a list
    if suff:
        root = funcy.remove(lambda x: x==suff[-1], cleaned_vals)
        return pd.Series(
            {'clean_first_name': root[0],
                 'clean_middle_name': ' '.join(root[1:]),
                    'clean_suffix': suff[-1]})
    return pd.Series(
        {'clean_first_name': cleaned_vals[0],
             'clean_middle_name': np.nan if len(cleaned_vals)==1 else ' '.join(cleaned_vals[1:]),
                'clean_suffix': np.nan})

In [6]:
cleaned_suffix_df = pd.DataFrame(concat_df.loc[~pd.isnull(concat_df.first_middle), 'first_middle'].apply(strip_first_middle))

In [7]:
def get_suffix(raw_str):
    # wrapper around get suffix
    if pd.isnull(raw_str) or len(raw_str.split(' ')) < 2:
        return np.nan
    str_list = raw_str.split(' ')
    suff = has_suffix(str_list)
    if suff:
        return suff[-1]
    return np.nan

In [8]:
# now we have all the associates, sep first middle into first and middle name, then sort and check 
# to see if we have any duplicates
df2 = pd.concat([concat_df, cleaned_suffix_df], axis=1)
df2.loc[:, 'clean_last_name'] = df2.lastname.apply(clean_names)

In [57]:
df2.head()

,citizenship,data_source,dno,dob,eod_year,first_middle,firstname,generation,institute,intern_dte,...,source,ssn,supervisor,unknown,year_grad,clean_first_name,clean_middle_name,clean_suffix,clean_last_name,len_middle
0,NaN,1964-1973 associates.XLS,117,NaN,1968.0,NaN,Robert,NaN,NIAMD,1966-1967,...,NIH EDUCATION,NaN,NaN,NaN,1966.0,NaN,NaN,NaN,ASHMAN,NaN
1,NaN,1964-1973 associates.XLS,119,NaN,1967.0,NaN,Philip,NaN,NIAMD,1965-1966,...,NIH EDUCATION,NaN,Decker,NaN,1965.0,NaN,NaN,NaN,ASKENASE,NaN
2,NaN,1964-1973 associates.XLS,120,NaN,1971.0,NaN,Floyd,NaN,NIGMS,1969-1970,...,NIH EDUCATION,NaN,Sjoerdsma,NaN,1969.0,NaN,NaN,NaN,ATKINS,NaN
3,NaN,1964-1973 associates.XLS,121,NaN,1965.0,NaN,Arthur,NaN,NIAID,1963-1964,...,NIH EDUCATION,NaN,NaN,NaN,1963.0,NaN,NaN,NaN,ATKINSON,NaN
4,NaN,1964-1973 associates.XLS,122,NaN,1971.0,NaN,John,NaN,NIAID,NaN,...,NIH EDUCATION,NaN,NaN,NaN,1969.0,NaN,NaN,NaN,ATKINSON,NaN


In [58]:
# consolidate firstname columns
missing_first_middle = pd.isnull(df2.first_middle)
df2.loc[missing_first_middle, ['clean_first_name', 'clean_middle_name']] = df2.loc[missing_first_middle, ['firstname', 'middlename']].applymap(clean_names)
df2.loc[:, 'len_middle'] = df2.clean_middle_name.apply(lambda x: np.nan if pd.isnull(x) else len(x.split(' ')))

# clean suffix out from any other columns
may_have_suffix_mask = df2.len_middle > 1
df2.loc[may_have_suffix_mask, 'clean_suffix'] = df2[may_have_suffix_mask]['clean_middle_name'].apply(get_suffix)

df3 = df2.drop(['first_middle', 'firstname', 'middlename', 'lastname', 'len_middle'], axis=1)
# df3 = df2
# # dropnow where both first and last name are missing
df4 = df3.dropna(subset=['clean_first_name', 'clean_last_name'], how='all')
print df3.shape
print df4.shape

(10729, 24)
(10723, 24)


In [59]:
# # remove all females
# female_mask = np.logical_or(
#     df4.clean_first_name.isin(FEMALE_FIRST_NAMES), df4.clean_middle_name.isin(FEMALE_MIDDLE_NAMES))

# df5 = df4.loc[~female_mask, :].sort_values(NAME_COLS)
df5 = df4.sort_values(NAME_COLS)

df5.loc[:, 'clean_middle_initial'] = df5.clean_middle_name.apply(lambda x: np.nan if pd.isnull(x) or x=='' else x[0])
df5.loc[:, 'clean_first_initial'] = df5.clean_first_name.apply(lambda x: np.nan if pd.isnull(x) or x=='' else x[0])
df5.loc[:, 'clean_medical_school'] = df5.med_school.apply(clean_med_school)

UNIVERSITY OF ATHENS MEDICAL SCHOOL IN ATHENS GREECE
UNIVERSITY OF ATHENS MEDICAL SCHOOL IN ATHENS GREECE
UNIVERSITY OF SAINT ANTOINE SCHOOL OF MEDICINE PARIS FRANCE
UNIVERSITY OF SAINT ANTOINE SCHOOL OF MEDICINE PARIS FRANCE


In [60]:
df5.loc[df5.med_school.apply(lambda x: False if pd.isnull(x) else 'France' in x)]

,citizenship,data_source,dno,dob,eod_year,generation,institute,intern_dte,intern_hos,lab_brch,...,supervisor,unknown,year_grad,clean_first_name,clean_middle_name,clean_suffix,clean_last_name,clean_middle_initial,clean_first_initial,clean_medical_school
2774,NaN,Associates alpha by institute.XLS,2623,NaN,1986.0,NaN,NCI,1980-1981,Creteil Hospital; France and St. Germain-en-La...,Pediatric,...,Pizzo,NaN,1980.0,MARY,NMI,NaN,MOUTOT,N,M,UNIVERSITY OF SAINT ANTOINE SCHOOL OF MEDICINE...
8590,NaN,Associates data.XLS,2623,NaN,1986.0,NaN,NCI,1980-1981,Creteil Hospital; France and St. Germain-en-La...,Pediatric,...,Pizzo,NaN,1980.0,MARY,NMI,NaN,MOUTOT,N,M,UNIVERSITY OF SAINT ANTOINE SCHOOL OF MEDICINE...
3842,NaN,Associates alpha by institute.XLS,240,NaN,1983.0,NaN,NIAID,1980-1981,University of Massachusetts,NaN,...,Rowe,NaN,1980.0,STEVEN,IRA,NaN,BENN,I,S,LA FACULTE DE MEDECINE D ANGERS FRANCE
6239,NaN,Associates data.XLS,240,NaN,1983.0,NaN,NIAID,1980-1981,University of Massachusetts,NaN,...,Rowe,NaN,1980.0,STEVEN,IRA,NaN,BENN,I,S,LA FACULTE DE MEDECINE D ANGERS FRANCE


In [61]:
def count_missing(row):
    # count the number of entries that are null
    return row[pd.isnull(row)].size

df5.loc[:, 'count_missing'] = df5[
    NAME_COLS+['clean_medical_school', 'eod_year', 'res_dtes', 'res_hosp', 'year_grad']].apply(count_missing, axis=1)

In [62]:
# drop duplicates by dno
df6 = df5.drop_duplicates('dno').sort_values(
    ['clean_last_name', 'clean_first_initial', 'clean_middle_name', 'count_missing'], axis=0)
print df5.shape
print df6.shape

(10723, 28)
(4112, 28)


In [63]:
man_raw = pd.read_excel(os.path.join(CORRECTIONS_DIR, 'manual_dno_matches.xlsx'), index=False).rename(columns={'medical_school': 'dno_medical_school'})
man_raw['medical_school'] = man_raw.dno_medical_school.apply(clean_med_school)
man = man_raw.drop(['application_year_max', 'application_year_min', 'dno_medical_school'], axis=1)


In [64]:
df_merge = df6.drop(NAME_COLS, axis=1)

In [65]:
df_  = pd.merge(left=df_merge, right=man, on=['dno'], how='inner')
print df_.shape
print man.shape
# all missing are femal
# overwrite medical school
df_['clean_medical_school'] = df_['medical_school']


(104, 29)
(104, 5)


In [66]:
df_.loc[df_.clean_medical_school!=df_.medical_school, NAME_COLS+['clean_medical_school', 'medical_school']]

,clean_first_name,clean_middle_name,clean_last_name,clean_medical_school,medical_school
38,RICHARD,A,JOHNSON,NaN,NaN
62,WILLIAM,M,MOORE,NaN,NaN


In [67]:
df6_ = df_.combine_first(df6)
print df_.shape
print df6.shape
print df6_.shape

(104, 29)
(4112, 28)
(4216, 29)


In [68]:
# reassign name of df6 and continue with merge
df6 = df6_

# need to fill in missing first, middle and last names

In [69]:
def get_year_diff(x):
    # get reference value
    ref_year = x.iloc[0]
    return pd.Series(x - ref_year)

def is_duplicate(grouped_df):
    # return true if grouped df should be counted as one duplicate, returns filter mask
    # check if med schools match/are missing
    # check if eod years match/are missing
    # first group_id
    if grouped_df.shape[0] == 1:
        return pd.Series([np.nan])
    eod_years = grouped_df['eod_year'].dropna().unique()
    if eod_years.shape[0] > 0:
        if max(eod_years) - min(eod_years) > 5:
            return pd.Series([np.nan]*grouped_df.shape[0])
    med_schools = grouped_df['clean_medical_school'].dropna().unique()
    # if only <= 1 unique med school, mark as duplicate
    ref_dno = grouped_df['dno'].values[0]
    if len(med_schools) < 2:
        return pd.Series([ref_dno]*grouped_df.shape[0])
    ms1 = med_schools[0]
    # otherwise, check the string similarity of the med school
    med_school_sims = [
        np.nan if pd.isnull(ms) else fuzz.partial_token_sort_ratio(
                ms1, ms) for ms in grouped_df.clean_medical_school.values]
    # for med schools with high string sim, count as dups, otherwise not 
    is_dups = [ref_dno if pd.isnull(sim) or sim > .8 else False for sim in med_school_sims]
    return pd.Series(is_dups)
    

In [70]:
df6.loc[df6.dno==4130, :]
df6.loc[df6.clean_last_name=='ZIVIN', :]
df4.loc[df4.clean_last_name=='COPELAND', :]

,citizenship,data_source,dno,dob,eod_year,generation,institute,intern_dte,intern_hos,lab_brch,...,residency,source,ssn,supervisor,unknown,year_grad,clean_first_name,clean_middle_name,clean_suffix,clean_last_name
319,NaN,1964-1973 associates.XLS,728,NaN,1973.0,NaN,NIEHS,1972-1973,Duke,NaN,...,NaN,NIH EDUCATION,NaN,NaN,NaN,1972.0,NaN,NaN,NaN,COPELAND
320,NaN,1964-1973 associates.XLS,729,NaN,1971.0,NaN,NHI,1969-1970,University Hospital of San Diego,NaN,...,Surgery,NIH EDUCATION,NaN,Morrow,NaN,1969.0,NaN,NaN,NaN,COPELAND
3280,NaN,Associates alpha by institute.XLS,729,NaN,1971.0,NaN,NHI,1969-1970,University Hospital of San Diego,NaN,...,Surgery,NIH EDUCATION,NaN,Morrow,NaN,1969.0,JACK,G,NaN,COPELAND
5046,NaN,Associates alpha by institute.XLS,728,NaN,1973.0,NaN,NIEHS,1972-1973,Duke,NaN,...,NaN,NIH EDUCATION,NaN,NaN,NaN,1972.0,DANA,DERWARD,NaN,COPELAND
6727,NaN,Associates data.XLS,728,NaN,1973.0,NaN,NIEHS,1972-1973,Duke,NaN,...,NaN,NIH EDUCATION,NaN,NaN,NaN,1972.0,DANA,DERWARD,NaN,COPELAND
6728,NaN,Associates data.XLS,729,NaN,1971.0,NaN,NHI,1969-1970,University Hospital of San Diego,NaN,...,Surgery,NIH EDUCATION,NaN,Morrow,NaN,1969.0,JACK,G,NaN,COPELAND


In [71]:
df6_grouped = df6.groupby(['clean_last_name', 'clean_first_initial'])
df6['eod_year_diff'] = df6_grouped.eod_year.apply(get_year_diff)
# df6['is_dup_index'] = df6_grouped.apply(is_duplicate)
new_col = df6_grouped.apply(is_duplicate)
# reset so no longer grouped as a multi index, then merge in via clean first, last_name

# dups = df6_grouped.filter(lambda x: is_duplicate(x)) 

In [72]:
new_col2 = new_col.reset_index()
new_col2.drop('level_2', axis=1, inplace=True)
print new_col.shape
print new_col2.shape


(4208,)
(4208, 3)


In [73]:
new_col2.columns = ['clean_last_name', 'clean_first_initial', 'duplicate_dno']
new_col3 = new_col2.drop_duplicates(['clean_last_name', 'clean_first_initial', 'duplicate_dno'])

In [74]:
new_col4 = new_col3[new_col3['duplicate_dno']!=False]
new_col4[new_col4['clean_last_name']=='AARON']

,clean_last_name,clean_first_initial,duplicate_dno
0,AARON,R,NaN


In [75]:
df7 = pd.merge(left=df6, right=new_col4, on=['clean_last_name', 'clean_first_initial'], how='inner')

In [76]:
df7.loc[df7.clean_last_name=='FRAM']

,citizenship,clean_first_initial,clean_first_name,clean_last_name,clean_medical_school,clean_middle_initial,clean_middle_name,clean_suffix,count_missing,data_source,...,res_dtes,res_hosp,residency,source,ssn,supervisor,unknown,year_grad,eod_year_diff,duplicate_dno


In [77]:
NONDUP_DNO = (187,  224,  244,  246,  256,  257,  345,  350,  349,  458,  461,
        474,  623,  643,  645,  665,  667,  673,  719,  807,  873,  996,
       1099, 1153, 1163, 1176, 1229, 1237, 1356, 1393, 1541, 1648, 1794,
       1798, 1801, 1882, 1913, 2032, 2058, 2105, 2173, 2183, 2189, 2207,
       2210, 2228, 2258, 2355, 2371, 2400, 2463, 2474, 2542, 2548, 2554,
       2559, 2596, 2653, 2655, 2672, 2866, 3021, 3138, 3140, 3172, 3207,
       3313, 3317, 3320, 3333, 3334, 3373, 3402, 3429, 3453, 3472, 3482,
       3484, 3485, 3490, 3497, 3498, 3510, 3600, 3644, 3646, 3710, 3721,
       3985, 3990, 3993, 4031, 4058, 4090, 4092, 4095)

df7.loc[:, 'eod_year_diff'] = df7.eod_year_diff.abs()
df7.loc[(df7.eod_year_diff>2) , 'duplicate_dno'] = np.nan
df7.loc[df7.dno.isin(NONDUP_DNO) , 'duplicate_dno'] = 0 
df7.loc[df7.duplicate_dno==df7.dno , 'duplicate_dno'] = np.nan

In [78]:
df7.loc[:, 'dup_flag'] = 0
dup_mask = df7.dno.isin(df7.duplicate_dno.dropna())
df7.loc[dup_mask, 'dup_flag'] = 1

In [79]:
# df7.loc[df7.clean_last_name.isin(['STEWART', "STEPHEN", 'BROWN']), NAME_COLS+['dno', 'dup_flag']]

In [80]:
missing_flags = ['missing_{}'.format(f) for f in df7.columns]

def count_missing(x):
    # returns 1 for missing value
    if x is None or x in ['None', '', ' ']:
        return 1
    return 1 if pd.isnull(x) else 0

# calculate missing info stats
df7[missing_flags] = df7.applymap(count_missing)

# calculate missing stats for people in the data set
df7[missing_flags].describe().to_csv(os.path.join(SUM_STAT_DIR, 'NIH_attendees_data_fill_rates.csv'))
df8 = df7.drop(missing_flags, axis=1)

In [81]:
def consolidate_row(series):
    nonnulls = series.dropna().unique()
    if nonnulls.shape[0] == 0:
        return pd.Series([np.nan]*series.shape[0])
    if isinstance(nonnulls[0], str):
        val = sorted(nonnulls, key=len, reverse=True)[0]
        return pd.Series([val]*series.shape[0])
    val = sorted(nonnulls, reverse=True)[0]
    return pd.Series([val]*series.shape[0])


In [82]:
grouped_dups = df8.loc[
    df8.duplicated(['clean_last_name', 'clean_first_initial'], keep=False), :].groupby(
        ['clean_last_name', 'clean_first_initial'], as_index=False)
new_df = []
for g, df in grouped_dups:
    consol_df = df.apply(consolidate_row, axis=0)
    consol_df['duplicate_dno'] = [df.dno.values]*df.shape[0]
    new_df.append(consol_df)
    

In [83]:
new_df2 = pd.concat(new_df, axis=0)
new_df3 = new_df2.drop_duplicates(['clean_first_name', 'clean_last_name', 'eod_year'])

In [84]:
missing = df6.loc[~df6.dno.isin(df8.dno),:].sort_values(['clean_last_name', 'clean_medical_school']).drop_duplicates(['clean_last_name', 'clean_medical_school'])
print missing.shape

(5, 30)


In [85]:
missing[NAME_COLS+['clean_medical_school', 'medical_school', 'eod_year']]

,clean_first_name,clean_middle_name,clean_last_name,clean_medical_school,medical_school,eod_year
25,DAVID,H,FRAM,YALE UNIVERSITY SCHOOL OF MEDICINE,YALE UNIVERSITY SCHOOL OF MEDICINE,1968.0
10548,NaN,NaN,FRANK,HARVARD MEDICAL SCHOOL,NaN,1962.0
26,BERNARD,LEONARD,FRANKEL,ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA...,ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA...,1970.0
27,ROBERT,NaN,FREEDMAN,HARVARD MEDICAL SCHOOL,HARVARD MEDICAL SCHOOL,1973.0
10551,NaN,NaN,FREIDMAN,NaN,NaN,1960.0


In [86]:
df8.loc[df8.duplicate_dno==0, 'duplicate_dno'] = np.nan

In [87]:
#now, merge in manual corrections to add any more duplicate dnos
# finally consolidate people merged into the same row
df9 = pd.concat([df8.loc[pd.isnull(df8.duplicate_dno), :], new_df3, missing], axis=0)


# df7.loc[df7.duplicated(['clean_last_name', 'clean_medical_school'], keep=False), NAME_COLS+['clean_medical_school', 'eod_year', 'dno', 'duplicate_dno']]

In [88]:
# df9.loc[df9.clean_last_name.isin(['STEWART', 'BROWN']), NAME_COLS]

In [89]:
df9.loc[df9.clean_last_name=='TEW', NAME_COLS+['dup_flag', 'duplicate_dno', 'dno']]
df8.loc[df8.dno==3699, NAME_COLS+['dup_flag', 'duplicate_dno', 'dno']]
df8.loc[df8.dno.isin([3700, 3699]), NAME_COLS+['dup_flag', 'duplicate_dno', 'dno']]
df8.loc[df8.dno.isin([3700, 3699]), NAME_COLS+['dup_flag', 'duplicate_dno', 'dno']]


# df9.loc[df9.duplicated('dno', keep=False), NAME_COLS+['dno', 'dup_flag', 'duplicate_dno']]

df9.loc[df9.clean_last_name=='FRAM', NAME_COLS+['dup_flag', 'duplicate_dno', 'dno']]

,clean_first_name,clean_middle_name,clean_last_name,dup_flag,duplicate_dno,dno
25,DAVID,H,FRAM,NaN,NaN,1153.0


In [90]:
to_remove = ['TERRECE', 'FRED', 'LAURENCE',
             'CUONO', 'DEFRENZE', 'JEFFERY', 'FINKLEMAN', 'SHERRAD', 'ANSCHNETZ', 'MARC', 'JENSON', 'KASTI', 
            'ADELBERT', 'RITCHARD', 'MANSFORD', 'DEFRENZO', 'DROBIN', 'HAMES', 'KREUZ', 'JERROLD', 'MANEUSI',
            'UNGARO']
to_replace = ['TERRENCE', 'FREDERICK', 'LAWRENCE',
              'CUOMO', 'DEFRONZO', 'JEFFREY', 'FINKELMAN', 'SHERRARD', 'ANSCHUETZ', 'MARCUS', 'JENSEN', 'KASTL',
              'ALBERT', 'RITCHARD', 'MANIFORD', 'DEFRONZO', 'DROBIS', 'JAMES', 'KRUEZ', 'JERROD', 'MANCUSI',
              'UNGARO']

correct_name_mispellings_fnc = funcy.rpartial(correct_mispellings, to_remove, to_replace)

df9.loc[:, 'clean_last_name'] = df9.clean_last_name.apply(correct_name_mispellings_fnc)
df9.loc[:, 'clean_first_name'] = df9.clean_first_name.apply(correct_name_mispellings_fnc)

In [91]:
# function to go in and correct some of the name mispellings in both data sets
# MUTATING FUNCTION
def change_names(df, selection_type, selection_value, to_change_type, to_change_values):
    for t, v in zip(to_change_type, to_change_values):
        print t, v
        df.loc[df[selection_type]==selection_value, t] = v

In [92]:
change_names(
    df9, 'clean_last_name', 'CHESEBRO', ['clean_first_name', 'clean_middle_name'], ['BRUCE', 'WILCOX'])
change_names(df9, 'clean_last_name', 'GALANTER', ['clean_first_name', 'clean_middle_name'], ['MARC', 'I'])
change_names(
    df9, 'clean_last_name', 'BEAN', ['clean_first_name', 'clean_middle_name', 'clean_medical_school'], ['SIDNEY', 'CHARLES', 'WAKE_FOREST'])
change_names(
    df9, 'clean_last_name', 'EILER', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])
change_names(
    df9, 'clean_last_name', 'FALCHUK', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])


clean_first_name BRUCE
clean_middle_name WILCOX
clean_first_name MARC
clean_middle_name I
clean_first_name SIDNEY
clean_middle_name CHARLES
clean_medical_school WAKE_FOREST
clean_first_name DONALD
clean_middle_name MARTIN
clean_first_name DONALD
clean_middle_name MARTIN


In [93]:
# fill in missing eod year
# manual corrections
df9.loc[(df9.clean_last_name=='BEELS') & pd.isnull(df9.eod_year), 'eod_year'] = 1967
df9.loc[(df9.clean_last_name=='KRAUSE') & pd.isnull(df9.eod_year), 'eod_year'] = 1965
df9.loc[(df9.clean_last_name=='MELLMAN') & pd.isnull(df9.eod_year), 'eod_year'] = 1986
# df9.loc[pd.isnull(df9.eod_year), :]

In [94]:
# import manual eod year fixes
manual_eod_fixes = pd.read_excel(os.path.join(CORRECTIONS_DIR, 'manual_eod_fixes.xlsx'))
manual_eod_fixes.head()
manual_eod_df = manual_eod_fixes[['clean_last_name'] + [c for c in manual_eod_fixes.columns if c.startswith('to_fix')]]

missing_eod = pd.merge(
    left=df9.loc[pd.isnull(df9.eod_year), :], right=manual_eod_df, on='clean_last_name', how='inner')

for x in ['clean_medical_school', 'clean_first_name', 'clean_middle_name', 'eod_year']:
    mask = ~pd.isnull(missing_eod['to_fix_{}'.format(x)])
    missing_eod.loc[mask, x] = missing_eod.loc[mask, 'to_fix_{}'.format(x)]


In [95]:
missing_eod2 = missing_eod.drop(
    (c for c in missing_eod.columns if c not in df9.columns), axis=1).drop_duplicates(['clean_last_name'])

In [96]:
df10 = pd.concat([df9.loc[~pd.isnull(df9.eod_year), :], missing_eod2], axis=0).sort_values(NAME_COLS, axis=0)
# drop duplicates on dno

In [97]:
df10A = df10.drop_duplicates('dno', keep='first')

In [98]:
dups = df10A.loc[df10A.duplicated(NAME_COLS+['clean_medical_school'], keep=False),:]
dups['keep'] = 1
dups.loc[pd.isnull(dups.duplicate_dno),'keep'] = 0
# NAME_COLS+['eod_year', 'dno', 'duplicate_dno']]
# df10A.loc[df10A.duplicated(NAME_COLS, keep=False), NAME_COLS+['duplicate_dno', 'dno']]

/home/lraymond/MIT/Azoulay_2016/yellow_berets/yb/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [99]:
non_dups = dups.sort_values(NAME_COLS+['eod_year', 'keep'], ascending=False).drop(['keep'], axis=1)
non_dups = non_dups.drop_duplicates(NAME_COLS+['eod_year'])

In [100]:
uniques = pd.concat([df10A.loc[~df10A.dno.isin(dups.dno)], non_dups]).drop_duplicates('dno')
print df10A.shape
print uniques.shape

(4063, 32)
(4046, 32)


In [102]:
lname = 'NAKASOTO'
# df10.loc[~pd.isnull(df10.duplicate_dno), NAME_COLS+['dno', 'duplicate_dno']]
# df10.loc[df10.clean_last_name.isin(['STEWART', 'BROWN', 'SHERWIN', 'YOUNGER']), NAME_COLS+['eod_year', 'dno', 'duplicate_dno', 'clean_medical_school']]
# uniques.loc[uniques.clean_last_name.isin(['STEWART', 'BROWN', 'SHERWIN', 'YOUNGER']), NAME_COLS+['eod_year', 'dno', 'duplicate_dno', 'clean_medical_school']]

In [103]:
# add columns keeping old names so we can check them
uniques['NIH_first_name'] = uniques.clean_first_name
uniques['NIH_middle_name'] = uniques.clean_middle_name
uniques['NIH_last_name'] = uniques.clean_last_name

In [104]:
# it seems that dno does refer to unique person, so drop dups based on that 
# save this unique to pick
uniques.to_pickle(os.path.join(PICKLE_DIR, 'unique_attendees.p'))

# to csv
uniques.to_csv(os.path.join(ATT_DATA_DIR, 'unique_attendees.csv'))

In [105]:
missing_eod_year = df10.loc[pd.isnull(df10.eod_year), :].sort_values('clean_last_name')

missing_eod_year[NAME_COLS+['clean_medical_school', 'dno']].to_csv(os.path.join(ATT_DATA_DIR, 'missing_eod_year.csv'), index=False)